Notebook structure to serve as a template for all other matches. Should read in data, synchronize event and tracking feeds then run pitch control for frames per event

In [1]:
%load_ext autoreload
%autoreload 2

In [119]:
import pandas as pd
import os
import json
import numpy as np
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
import matplotsoccer as mps
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import pandas as pd
from kloppy import secondspectrum
import pickle
import sys
 
# setting path
sys.path.append('../')
 
# # importing
from src.second_spectrum_utils import get_home_away_tracking
import src.player_velocities as vel
import src.pitch_control as pc
import src.visualizations as viz

import src.statsbomb_util as sb

In [120]:
META_PATH = "../man-city-data/SecondSpectrum/MCI Women_s Files-001/g2312213_SecondSpectrum_meta.xml"
RAW_PATH = "../man-city-data/SecondSpectrum/MCI Women_s Files-001/g2312213_SecondSpectrum_tracking-produced.jsonl"
EVENT_PATH = "../man-city-data/StatsBomb/Data/ManCity_Tottenham_events.json"

PARAMS = pc.parameters()

In [71]:
dataset = secondspectrum.load(
    meta_data=META_PATH,
    raw_data=RAW_PATH,
    
    # Optional arguments
    sample_rate=1/1,
    coordinates="secondspectrum",
    only_alive=False)

In [140]:
dataset_alive = secondspectrum.load(
    meta_data=META_PATH,
    raw_data=RAW_PATH,
    
    # Optional arguments
    sample_rate=1/1,
    coordinates="secondspectrum",
    only_alive=True)

In [73]:
home_df, away_df = get_home_away_tracking(dataset)
tracking_home = vel.calc_player_velocities(home_df,smoothing=True,filter_='moving_average')
tracking_away = vel.calc_player_velocities(away_df,smoothing=True,filter_='moving_average')

In [157]:
tracking_df = dataset_alive.to_df()

In [158]:
tracking_df = tracking_df[['period_id','timestamp','frame_id','ball_state','ball_owning_team_id']]
tracking_df = tracking_df.rename(columns={'timestamp': 'time_sec'})

In [159]:
event_data = pd.read_json(EVENT_PATH)
event_df = sb.rename_cols(sb.normalize_dataframe(event_data))

In [160]:
event_df = sb.normalize_time_column(event_df)

In [161]:
event_df = event_df.dropna(subset = ['obv_total_net'])
event_df = event_df.loc[event_df['duration'] != 0]
event_df.head()

,event_id,index,period_id,timestamp,minute,second,possession,obv_for_after,obv_for_before,obv_for_net,...,substitution_replacement_id,substitution_replacement_name,miscontrol_aerial_won,pass_cut_back,injury_stoppage_in_chain,shot_deflected,block_deflection,timestamp_shift,timediff,time
4,b81dcf83-537d-4f17-8f34-abb38d2dce13,5,1,1681948800194,0,0,2,0.016658,0.006829,0.009830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.681949e+12,0.194,0.19
6,45e144b9-05dc-4150-a4a9-c0662f6488d7,7,1,1681948801037,0,1,2,0.017105,0.016658,0.000447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.681949e+12,0.000,1.04
7,621b03c3-79ce-48f6-a123-4ff7f92b0201,8,1,1681948801691,0,1,2,0.014085,0.017105,-0.003020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.681949e+12,0.654,1.69
9,de02e35f-0740-481c-8e31-d6dd268a8377,10,1,1681948802903,0,2,2,0.014085,0.014085,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.681949e+12,0.000,2.90
10,2f318aa7-799f-48ba-b8ca-3015600cb591,11,1,1681948803965,0,3,2,0.012647,0.014085,-0.001438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.681949e+12,1.062,3.96


In [162]:
t1h = tracking_df[(tracking_df.period_id == 1) & (tracking_df.ball_owning_team_id == 'home')]
t1a = tracking_df[(tracking_df.period_id == 1) & (tracking_df.ball_owning_team_id == 'away')]

t2h = tracking_df[(tracking_df.period_id == 2) & (tracking_df.ball_owning_team_id == 'home')]
t2a = tracking_df[(tracking_df.period_id == 2) & (tracking_df.ball_owning_team_id == 'away')]

e1h = event_df[(event_df.period_id == 1) & (event_df['team_id'] == 746)]
e1a = event_df[(event_df.period_id == 1) & (event_df['team_id'] != 746)]
e2h = event_df[(event_df.period_id == 2) & (event_df['team_id'] == 746)]
e2a = event_df[(event_df.period_id == 2) & (event_df['team_id'] != 746)]

In [163]:
merged_1 = pd.merge_asof(e1h.sort_values('time'), t1h.sort_values('time_sec'), 
                           left_on='time', right_on='time_sec', 
                           direction='nearest')
merged_2 = pd.merge_asof(e1a.sort_values('time'), t1a.sort_values('time_sec'), 
                           left_on='time', right_on='time_sec', 
                           direction='nearest')
merged_3 = pd.merge_asof(e2h.sort_values('time'), t2h.sort_values('time_sec'), 
                           left_on='time', right_on='time_sec', 
                           direction='nearest')
merged_4 = pd.merge_asof(e2a.sort_values('time'), t2a.sort_values('time_sec'), 
                           left_on='time', right_on='time_sec', 
                           direction='nearest')
merged_df = pd.concat([merged_1,merged_2,merged_3,merged_4])

In [164]:
merged_df.to_csv("city_tottenham_merged.csv")

In [165]:
merged_df.shape

(1999, 151)